In [ ]:
#排程code
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from bs4 import BeautifulSoup
import pymssql


chrome_path='C:\\Users\\admin\\Desktop\\專題\\chromedriver.exe'
browser = webdriver.Chrome(chrome_path)
browser.get("http://www.sitca.org.tw/ROC/Industry/IN2629.aspx?pid=IN22601_04")
#開啟chrome到目標網頁
conn=pymssql.connect(host='163.18.42.30',user='root',password='1234',database='fund',charset='utf8')
cursor=conn.cursor()
#連結資料庫

sql_drop=r'''
drop table fund_schedule_input
'''
cursor.execute(sql_drop)
sql_create=r'''
CREATE TABLE fund_schedule_input(
	[time] [nvarchar](50) NULL,
	[company] [nvarchar](50) NULL,
	[fundName] [nvarchar](50) NULL,
	[rank] [nvarchar](50) NULL,
	[subjectKind] [nvarchar](50) NULL,
	[subjectCode] [nvarchar](50) NULL,
	[subjectName] [nvarchar](50) NULL,
	[price] [nvarchar](50) NULL,
	[proportion] [nvarchar](50) NULL
) ON [PRIMARY]
'''
cursor.execute(sql_create)
conn.commit()
#重製表

time.sleep(3)        

combobox_time=Select(browser.find_element_by_id("ctl00_ContentPlaceHolder1_ddlQ_YM"))
c_tLen=len(combobox_time.options)
#combobox年月
combobox_company=Select(browser.find_element_by_id("ctl00_ContentPlaceHolder1_ddlQ_Comid1"))
c_cLen=len(combobox_company.options)
#combobox公司
for A in range(c_tLen-1,c_tLen):
    combobox_time=Select(browser.find_element_by_id("ctl00_ContentPlaceHolder1_ddlQ_YM"))
    combobox_time.select_by_index(A)#
    time.sleep(5)
    combobox_time=Select(browser.find_element_by_id("ctl00_ContentPlaceHolder1_ddlQ_YM"))
    yearmonth=combobox_time.first_selected_option.text
    print(yearmonth+'\n')
    for B in range(c_cLen):
        radio=browser.find_element_by_id("ctl00_ContentPlaceHolder1_rbComCL")
        radio.click()
        #radio公司+類型選取
        combobox_stuck=Select(browser.find_element_by_id("ctl00_ContentPlaceHolder1_ddlQ_Class1"))
        combobox_stuck.select_by_index(1)
        #combobox國內股票型選取
        combobox_company=Select(browser.find_element_by_id("ctl00_ContentPlaceHolder1_ddlQ_Comid1"))
        combobox_company.select_by_index(B)#
        company=combobox_company.first_selected_option.text
        print(company)
        #combobox公司選取
        button_query=browser.find_element_by_id("ctl00_ContentPlaceHolder1_BtnQuery")
        button_query.click()
        #button查詢點擊
        time.sleep(5)
        bs = BeautifulSoup(browser.page_source,'html.parser')
        #取得網頁資訊
        #'html.parser'解析大寫html
        tr=bs.find_all('table')[3].find_all('tr')
        goal=[]
        data=[]
        row=[]
        for i in range(1,len(tr)):
            td=tr[i].find_all('td')
            for m in range(len(td)):
                if(len(td)==10):
                    if(m!=6 and m!=7 and m!=8):
                        goal.append(td[m].text)
                elif(len(td)==9):
                    if(m!=5 and m!=6 and m!=7):
                        goal.append(td[m].text)
            for j in range(len(td)):
                rowspan=td[j].get("rowspan")
                if(rowspan!=None):
                    row.append(int(rowspan))
        count=0
        Gjin=[]
        for i in range(len(row)):
            data.append([])
            Gjin.append(goal[count])
            for j in range(row[i]*6):
                count+=1
                data[i].append([])
                data[i][j].append(goal[count])
            count+=1
        #print(data)
        dic={}
        count=3
        dic['0']=yearmonth
        dic['1']=company
        for i in range(len(data)):
            for j in range(row[i]*6):
                dic['2']=Gjin[i]  
                dic[str(count)]=data[i][j]
                count+=1
                if(count==9):
                    count=3
                    sql="insert into fund_schedule_input values(%(0)s,%(1)s,%(2)s,%(3)s,%(4)s,%(5)s,%(6)s,%(7)s,%(8)s)"
                    cursor.execute(sql,dic)
                    conn.commit()

        #存入資料庫


 

        print('\n')
        time.sleep(1)


sql_drop=r'''drop table fund_schedule_cast'''
cursor.execute(sql_drop)
sql_create=r'''SELECT cast([subjectName] as nvarchar(50))as [subjectName]
      ,cast(replace(replace(replace ([time] ,' ',''),'年','-'),'月','')as nvarchar(30))as [time]
      ,SUM(cast(replace([price], ',', '') as money)/1000) as [price]
into fund_schedule_cast
FROM fund_schedule_input
group by subjectName,time'''
cursor.execute(sql_create)



sql_addColumn=r'''alter table fund_schedule_cast ADD marketShare float
alter table fund_schedule_cast ADD growthRate float
alter table fund_schedule_cast ADD Quadrant nvarchar(10)'''
cursor.execute(sql_addColumn)

sql_drop=r'''drop table fund_schedule_BCGcompute'''
cursor.execute(sql_drop)
sql_BCGcompute=r'''select *
into fund_schedule_BCGcompute
from
(
select time,subjectName,price,marketShare,growthRate,Quadrant from fund_schedule_cast 
UNION
select B.time,B.subjectName,B.price,B.marketShare,B.growthRate,B.Quadrant
from fund_schedule_cast A,fund_gapminder_new B
where cast(dateadd(month,-1,cast(A.time+'-01' as date)) as char(7))=B.time
)a'''
cursor.execute(sql_BCGcompute)

#計算相對市占率及成長率
sql_execSQL=r'''exec updateMarketShare
                exec updateGrowthRate'''
cursor.execute(sql_execSQL)
#添加至BCG_BCG
sql_execSQL=r'''insert into BCG_BCG (time,marketShare,growthRate)
select distinct time,AVG(marketShare),AVG(growthRate)
from fund_schedule_BCGcompute
where time=(select top 1 time from fund_schedule_BCGcompute order by time desc)
group by time
'''
cursor.execute(sql_execSQL)
#計算BCG象限
sql_execSQL=r'''exec updateBCG_Value'''
cursor.execute(sql_execSQL)
#刪除上個月資料
sql_execSQL=r'''delete from fund_schedule_BCGcompute
--搜尋最新時間-1個月得出的時間
where time=cast(dateadd(month,-1,
cast((select top 1 time from fund_schedule_BCGcompute order by time desc)+'-01' as date)) as char(7))'''
cursor.execute(sql_execSQL)
#合併至website
sql_execSQL=r'''select * into TEMP
from(
select [time]
      ,[subjectName]
      ,[price]
      ,[marketShare]*100 as marketShare
      ,[growthRate]*100 as growthRate
      ,[Quadrant] from fund_schedule_BCGcompute
union
select * from websiteShow
)a

drop table websiteShow
select * into websiteShow from TEMP
drop table TEMP '''
cursor.execute(sql_execSQL)
#合併至fund_gapminder_export
sql_execSQL=r'''
select * into temp
from fund_gapminder_export
union
SELECT [subjectName]
      ,[time]
      ,[marketShare]*100 as marketShare
      ,[growthRate]*100 as growthRate
      ,[price]
      ,[Quadrant]
  FROM [fund].[dbo].[fund_schedule_BCGcompute]
  order by 時間 desc
  
  drop table fund_gapminder_export
  select * into fund_gapminder_export
  from temp
  drop table temp'''
cursor.execute(sql_execSQL)
#添加hot
sql_execSQL=r'''
select [標的名稱]
      ,[時間]
      ,[市佔率]
      ,[成長率]
      ,[金額]
into temp 
from
(select [時間]
      ,標的名稱
      ,[金額]
      ,[市佔率]
      ,[成長率]
      ,row_number() over(partition by 時間 order by 金額 desc) 'rn'
 from [fund].[dbo].[fund_gapminder_export]) t where t.rn<=3 
 drop table hot
 select * into hot from temp
 drop table temp
 '''
cursor.execute(sql_execSQL)


conn.commit()
